# 实验2：VGG16猫狗图像分类

### 18124399冯秦萱

导入包，使用tensorflow中的keras实现vgg16网络的构建

In [ ]:
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2
import zipfile
from keras import optimizers
from keras import regularizers
from keras.optimizers import SGD
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

定义变量，图片宽高，batchsize和训练次数

In [ ]:
weight,height = 224,224
batch=10
epochs=100

加载猫狗数据集

In [ ]:
path_= "../input/dogs-vs-cats/"
with zipfile.ZipFile(path_+ "train.zip","r") as z:
    z.extractall(".")#数据集位置    
path = "/kaggle/working/train/"

通过opencv进行图片预处理，加载图片数据并设置标签，猫的标签为0，狗的标签为1

In [ ]:
def load_data():
    train_data = np.empty((5000,weight,height, 3), dtype="int32")
    train_label = np.empty((5000, ), dtype="int32")

    for i in range(5000):
        if i % 2:
            train_data[i] = cv2.resize(cv2.imread(path+'/'+'dog.' + str(i) + '.jpg'), (weight,height))
            train_label[i] = 1
        else:
            train_data[i] = cv2.resize(cv2.imread(path+'/' +'cat.' + str(i) + '.jpg'), (weight,height))
            train_label[i] = 0

    return train_data, train_label

定义vgg16网络结构，使用BN，dropout，L2正则化等进行优化

In [ ]:
def vgg16():
    weight_decay = 0.0005
    # layer1
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same',
                     input_shape=( weight,height, 3), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    # layer2
    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # layer3
    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    # layer4
    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # layer5
    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    # layer6
    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    # layer7
    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # layer8
    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    # layer9
    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    # layer10
    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # layer11
    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    # layer12
    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    # layer13
    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    # layer14
    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    # layer15
    model.add(Dense(512, kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    # layer16
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))
 
    return model

规范训练数据，对标签one-hot编码

In [ ]:
train_data, train_label = load_data()
train_data = train_data.astype('float32')
train_label = keras.utils.to_categorical(train_label, 2)#one-hot编码

展示vgg16模型结构

In [ ]:
model = vgg16()
model.summary()

定义损失函数，优化器

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) #设置优化器为SGD
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

模型训练，训练100次，训练集的20%作为验证集，shuffle=True打乱数据

In [ ]:
history = model.fit(train_data, train_label,
              batch_size=batch,
              epochs=epochs,
              validation_split=0.2,  #把训练集中的五分之一作为验证集
              shuffle=True)


保存模型及训练数据

In [ ]:
model.save('/kaggle/working/vgg16dogcat.h5')
data=pd.DataFrame(history.history)
data.to_csv('/kaggle/working/log.csv',index=False)

画图展示训练过程中精度和loss变化

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


epochs_range = range(epochs)


plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc,label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


测试模型

In [ ]:
 def judge(predicted):
    predicted = np.argmax(predicted)
    if predicted == 0:
        print('cat')
    else:
        print('dog')
load_model = load_model("/kaggle/working/vgg16dogcat.h5")  # 读取模型
img = cv2.imread('../input/testpng/cat.png')  
plt.imshow(img)
img = cv2.resize(img, (weight,height))
img = img.reshape(-1,weight,height,3)
img = img.astype('float32')
predicted = load_model.predict(img)  # 输出预测结果
print(predicted)
judge(predicted)